In [2]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPooling1D
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.layers import Reshape, Lambda
from tensorflow.keras.layers import Add, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adadelta, Adagrad, Adamax, Nadam
import numpy as np
from imutils import paths
import cv2
import random
import os
import tensorflow as tf
tf.executing_eagerly()

True

In [6]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    loss = K.ctc_batch_cost(labels, y_pred, input_length, label_length)
    # tf.print(y_pred.shape)
    # tf.print(labels.shape)
    # tf.print(tf.math.argmax(y_pred, axis=2)[0], summarize=-1, output_stream='file://pred.out')
    # tf.print(labels[0], summarize=-1, output_stream='file://label.out')
    return loss

In [11]:
def get_model(training,img_w,img_h,depth,alphabetLength,absolute_max_string_len):
    conv_filters = 16
    kernel_size = (3, 3)
    pool_size = 2
    time_dense_size = 32
    rnn_size = 512

    act='relu'
    if K.image_data_format()=='channels_first':
      input_shape=(depth,img_w,img_h)
    else:
      input_shape=(img_w,img_h,depth)

    input_data=Input(name='the_input',shape=input_shape,dtype='float32')
    
    base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=input_data)
    for l in base_model.layers:
        l.trainable = False

    inner = Reshape(target_shape=(32,2856), name='reshape')(base_model.get_layer('mixed_6a').output)
    inner = Dense(time_dense_size, activation=act, name='dense1')(inner)       

    gru_1 = GRU(rnn_size, return_sequences=True,
          kernel_initializer='he_normal', name='gru1')(inner)
    gru_1b = GRU(rnn_size, return_sequences=True,
           go_backwards=True, kernel_initializer='he_normal',
           name='gru1_b')(inner)
    gru1_merged = Add()([gru_1, gru_1b])

    gru_2 = GRU(rnn_size, return_sequences=True,
          kernel_initializer='he_normal', name='gru2')(gru1_merged)
    gru_2b = GRU(rnn_size, return_sequences=True, go_backwards=True,
           kernel_initializer='he_normal', name='gru2_b')(gru1_merged)

    # transforms RNN output to character activations:
    inner = Dense(alphabetLength, kernel_initializer='he_normal',name='dense2')(Concatenate()([gru_2, gru_2b]))
    
    y_pred = Activation('softmax', name='softmax')(inner)

    labels = Input(name='the_labels', shape=(absolute_max_string_len,), dtype='float32')
    input_length = Input(name='input_length', shape=(1,), dtype='int64')
    label_length = Input(name='label_length', shape=(1,), dtype='int64')

    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
    
    if training:
        return Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)
    else:
        return Model(inputs=[input_data], outputs=y_pred)

In [12]:
get_model(True,145,220,3,64,16).summary()

________________
activation_463 (Activation)     (None, 15, 25, 32)   0           batch_normalization_463[0][0]    
__________________________________________________________________________________________________
conv2d_461 (Conv2D)             (None, 15, 25, 32)   10240       block35_7_ac[0][0]               
__________________________________________________________________________________________________
conv2d_464 (Conv2D)             (None, 15, 25, 48)   13824       activation_463[0][0]             
__________________________________________________________________________________________________
batch_normalization_461 (BatchN (None, 15, 25, 32)   96          conv2d_461[0][0]                 
__________________________________________________________________________________________________
batch_normalization_464 (BatchN (None, 15, 25, 48)   144         conv2d_464[0][0]                 
____________________________________________________________________________________________

In [31]:
x = tf.zeros([32,75,75,3])
model = tf.keras.applications.InceptionResNetV2(weights='imagenet', input_shape=(75,75,3), include_top=False)
model.outputs = [model.get_layer('mixed_6a').output]


In [30]:
model = tf.keras.applications.InceptionResNetV2(weights='imagenet', input_shape=(75,75,3), include_top=False)
model.get_layer('mixed_6a').output

<tf.Tensor 'mixed_6a_12/Identity:0' shape=(None, 3, 3, 1088) dtype=float32>

In [43]:
@tf.function
def run_model(x):
    model = Dense(3, activation='relu', name='dense1')
    x = model(x)
    for i in range(3):
        x = Dense(3, activation='relu', name='dense1')(x)
    return x
    


x = tf.zeros([32,75,75,3])
run_model(x)



ValueError: in converted code:

    <ipython-input-34-97704c5aa866>:4 run_model  *
        x = model(x)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/base_layer.py:748 __call__
        self._maybe_build(inputs)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/base_layer.py:2116 _maybe_build
        self.build(input_shapes)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/layers/core.py:1113 build
        trainable=True)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/base_layer.py:446 add_weight
        caching_device=caching_device)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/training/tracking/base.py:744 _add_variable_with_custom_getter
        **kwargs_for_getter)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/engine/base_layer_utils.py:142 make_variable
        shape=variable_shape if variable_shape else None)
    /usr/l

In [ ]:
cust = CustomRNN()